In [2]:
#get dataframe of tweet_id and user_id
import pandas
import csv

df = pandas.read_csv('gamergate_tweets.csv')

In [3]:
import sys
sys.path.append("/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages")

from twython import Twython

#get first 1000 tweet_ids
CONSUMER_KEY = "48r56upQvsnTiSDfY4CNwIpCQ"
CONSUMER_SECRET = "VqZhFW7GqogdKXJnpuS0umVe211bFgoCdaQlRdRMayvrbdDh3k"
OAUTH_TOKEN = "18000528-Scl8a1pLinqP4KWtaa3f8EN0M1vqAGTyYXMtsLIGm"
OAUTH_TOKEN_SECRET = "NcIvLeqVbwwBegabvo7946FVIaDN0CAaJ7aS2aV7pZFzC"

twitter = Twython(
    CONSUMER_KEY, CONSUMER_SECRET,
    OAUTH_TOKEN, OAUTH_TOKEN_SECRET)


start = 0
end = 100
tweet_list = []
while end <= 5000:
    id_list = df["tweet_id"].tolist()
    current = id_list[start:end]
    tweet = twitter.lookup_status(id=current)
    start += 100
    end += 100
    for t in tweet:
        each_tweet = []
        each_tweet.append(t['user']['id'])
        each_tweet.append(t["retweet_count"])
        each_tweet.append(t["favorite_count"])
        if t["entities"]['urls'] != []:
            each_tweet.append(1)
        else: each_tweet.append(0)
        if t["entities"]["user_mentions"] == []:
            each_tweet.append([])
        else: 
            y = 0
            mentionlist = []
            while y < len(t["entities"]["user_mentions"]):
                mentionlist.append((t["entities"]["user_mentions"])[y].items()[2][1])
                y+=1
            each_tweet.append(mentionlist)
        each_tweet.append(t["text"])
        tweet_list.append(each_tweet)

In [4]:
#put into df    
tweets_df = pandas.DataFrame(tweet_list, columns=["User ID", "Retweets", "Favorites", "Has_url", "Mentions", "Text"])

In [5]:
#use lexicon to get polarity of each tweet

#read and put positive words into list
with open('positive-words.txt') as f:
    positive = f.read().splitlines()
#trim list    
positive = positive[35:]
positive = set(positive)

#read and put negative words into list
with open('negative-words.txt') as f:
    negative = f.read().splitlines()
#trim list    
negative = negative[35:]
#remove problematic "naive"
negative.remove(negative[3007])
negative = set(negative)
    
lexrating = []
poscount = 0
negcount = 0

x = 0

for tweet in tweets_df["Text"]:
    words = tweet.split()
    for word in words:
        if word in positive:
            poscount = poscount + 1
        elif word in negative:
            negcount = negcount + 1
            
    #get polarity of tweet
    if negcount == 0: ratio = poscount
    elif poscount == 0: ratio = negcount * -1
    else : 
        if poscount > negcount: ratio = poscount - negcount
        else: ratio = (negcount - poscount) * -1
    
    lexrating.append(ratio)

    poscount = 0
    negcount = 0
    x += 1
        
    #add column of lexrating reviews
    tweets_df["Polarity"] = pandas.Series(lexrating)
    

polaritylist = tweets_df["Polarity"].tolist()
likelist = tweets_df["Favorites"].tolist()
retweetlist = tweets_df["Retweets"].tolist()

#Create Popularity column, where popularity = num(Retweets) + num(Favorites)
x = 0
popularity = []
while x < len(likelist):
    popularity.append(likelist[x] + retweetlist[x])
    x+=1

tweets_df["Popularity"] = pandas.Series(popularity)

print tweets_df.sort(["Polarity"])
    

         User ID  Retweets  Favorites  Has_url  \
2525  1498976455         0          0        0   
2837    16088388         0          0        0   
1469  2368624716       192          0        0   
3377  1317712621       192          0        0   
1774  2801289353       192          0        0   
1209  2842420057         0          1        0   
2602  2801890131        88          0        0   
1785  2799282698         0          1        0   
1719   308407039        66         55        0   
639   1132062200         1          0        0   
2779  1727369563         0          0        0   
950    453427796       160          0        0   
3058  2839887204       160          0        0   
744   2733723912       160          0        0   
1452    16210842        88          0        0   
615   2812362302         1          1        0   
614   2814998691       160          0        0   
2639  1947709525        88          0        0   
1558  2614352281       160          0        0   


/Users/Chelsea/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:63: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [17]:
%run Get\ Users.ipynb
print users_df

         User ID  Follower Count  Favourites Count
1997  1004731904          224541              7272
328     26258609          105322             43651
550    308407039           89122             11851
1557    22655375           89122             11851
778   2774051122           86765              1585
189     26258609           70361             26244
1359    87329702           70361             26244
2710   992871366           70361             26244
3546  2790540392           70361             26244
229   2756873076           60101              1315
1389   162572003           57425              2831
2084   775543549           55451              2707
2096  2772953185           55451              2707
2492   992871366           55451              2707
2933   992871366           52262             17320
2460  2549103608           39641             31840
3383   162572003           39641             31840
2567   992871366           38144             11402
2779  2787101798           3814

In [30]:
#full_df = pandas.merge(tweets_df, users_df, on='User ID')
#print full_df.drop_duplicates